<a href="https://colab.research.google.com/github/AllenWangYi/test/blob/main/%E2%80%9Cmecs6616_Spring2023_Project3_yw3956_ipynb%E2%80%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Important***

**Before starting, make sure to read the [Assignment Instructions](https://courseworks2.columbia.edu/courses/172081/pages/assignment-instructions) page on Courseworks2 to learn the workflow for completing this project.**

**Different from Projects 1 and 2**, apart from the link to your notebook, you are also required to submit the collected data file `data.pkl` and your chosen model checkpoint `dynamic.pth` to Coursework. You should put the link to your notebook in the "Comment" section of your submission.

# Project Setup


In [ ]:
# DO NOT CHANGE

# After running this cell, the folder 'mecs6616_sp23_project3' will show up in the file explorer on the left (click on the folder icon if it's not open)
# It may take a few seconds to appear
!git clone https://github.com/roamlab/mecs6616_sp23_project3.git

Cloning into 'mecs6616_sp23_project3'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 27 (delta 11), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (27/27), 23.05 KiB | 357.00 KiB/s, done.


In [ ]:
# DO NOT CHANGE

# Copy all needed files into the working directory. This is simply to make accessing files easier
!cp -av /content/mecs6616_sp23_project3/* /content/

'/content/mecs6616_sp23_project3/arm_dynamics_base.py' -> '/content/arm_dynamics_base.py'
'/content/mecs6616_sp23_project3/arm_dynamics_teacher.py' -> '/content/arm_dynamics_teacher.py'
'/content/mecs6616_sp23_project3/geometry.py' -> '/content/geometry.py'
'/content/mecs6616_sp23_project3/imgs' -> '/content/imgs'
'/content/mecs6616_sp23_project3/imgs/example.png' -> '/content/imgs/example.png'
'/content/mecs6616_sp23_project3/render.py' -> '/content/render.py'
'/content/mecs6616_sp23_project3/robot.py' -> '/content/robot.py'
'/content/mecs6616_sp23_project3/score.py' -> '/content/score.py'


In [ ]:
# DO NOT CHANGE

# Install required packages
!pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 32.6 MB/s eta 0:00:00


# Starter Code Explanation

This project uses two 3-link arms, one called arm_teacher (blue) and the other called arm_student (red), as shown in the image below. For each test, a constant torque will be applied to the first joint of both arms for 5 seconds. arm_teacher is moving according to the provided ground truth forward dynamics and your job is to use deep learning to train the arm_student to learn the forward dynamics of the arm_teacher so that it can imitate its behavior. The forward dynamics is a function that takes in the current state of and an action applied to the arm, and then computes the new state of the arm. This project uses a time step of 0.01 second, meaning each time we advance the simulation, we compute the forward dynamics for 0.01 second. In the example image, the student arm is not updating its state and remains static but we will make it move after training is done.



<div>
<img src="https://github.com/roamlab/mecs6616_sp23_project3/blob/master/imgs/example.png?raw=true" width="600"/>
</div>

The interface for controlling the robot is defined in the `Robot` class in `robot.py` file. Each robot is initialized with a corresponding forward dynamics (the base class for forward dynamics definition is in `arm_dynamic_base.py`). The arm_teacher is initialized with the provided ground truth forward dynamics, as defined in `arm_dynamics_teacher.py`. You are welcome to look in-depth into this file to understand how the ground truth forward dynamics is computed for an arm, given its number of links, link mass, and viscous friction of the environment - this is recommended but not necessary to successfully complete this assignment. The state of each arm is defined with a (6,1)-dimensional numpy array (three joint positions in radians + three joint velocities in radians per second). An action is defined as the three toques (in Nm) applied to the three joints respectively, which is a (3,1) numpy array. **Throughout this project, we make the problem simpler by only applying a torque to the first joint, so the actions always look like `[torque,0,0]`.** Also, when scoring your model the robot will always start off in a hanging position, meaning an initial state of `[-pi/2,0,0,0,0,0]` so if the collected data from part 1 looks similar, the model will perform better. The `robot.py` file provides you with some functions to set/get the state and set the action for the arm. Make sure you understand `robot.py` well enough before getting started.

`geometry.py` provides some geometry functions and `render.py` defines how the visualization is rendered. These two files are not of particular interest for completing this project.

# Part I. Data collection.

You will first need to complete the cell below to collect a dataset for training the forward dynamics. After running the cell, it should generate a pickle file `data.pkl` that contains a data dictionary `data = {'X': X, 'Y': Y}`. The shape of `data['X']` should be (`num_samples`, 9), the first 6 elements are state and the last 3 elements are the action. The shape of `data['Y']` should be (`num_samples`, 6), which saves the next state after applying the action using the ground truth forward dynamics of arm_teacher.

**After the data file is generated, `data.pkl` should appear under the 'Files' icon in the left sidebar. You can download this file by right clicking the file name. You are required to submit this file. Please do not change its name.**

In the cell below, we have provided a minimal example of simulating the arm_teacher for 5 seconds. The GUI visualization is turned on and you should see the behavior of arm_teacher. The visualization can drastically slow down the simulator and you should turn it off when collecting a large amount of data.





In [ ]:
import numpy as np
import os
from arm_dynamics_teacher import ArmDynamicsTeacher
from robot import Robot
import pickle
import math
from render import Renderer
import time
np.random.seed(10)
# DO NOT CHANGE
# Teacher arm
dynamics_teacher = ArmDynamicsTeacher(
    num_links=3,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01
)
arm_teacher = Robot(dynamics_teacher)

# ---
# You code starts here. X and Y should eventually be populated with your collected data
# Control the arm to collect a dataset for training the forward dynamics.
X = np.zeros((300000, arm_teacher.dynamics.get_state_dim() + arm_teacher.dynamics.get_action_dim()))
Y = np.zeros((300000, arm_teacher.dynamics.get_state_dim()))

# We run the simulator for 5 seconds with a time step of 0.01 second,
# so there are 500 steps in total
num_steps = 300000

# GUI visualization, this will drastically reudce the speed of the simulator!
gui = False

# Define the initial state of the robot, such that it is vertical
initial_state = np.zeros((arm_teacher.dynamics.get_state_dim(), 1))  # position + velocity
initial_state[0] = -math.pi / 2.0

# Set the initial state of the arm. Input to set_state() should be of shape (6, 1)
arm_teacher.set_state(initial_state)

# Define the action, applying 1Nm torque to the first joint
action = np.zeros((arm_teacher.dynamics.get_action_dim(), 1))
action1 = np.random.uniform(-1.5, 1.5, 50)
action1 = np.repeat(action1, 500)
torques = np.random.uniform(0.5, 1.5, 50)
action2 = np.array([np.linspace(0, torque, 500) for torque in torques]).flatten()
action3 = np.random.uniform(-1.5, 1.5, 100000)
action_set = np.concatenate((action1,action2,action1,action2,action1,action2,action1, action1, action3))
# action[0] = 1

# Set the action. Input to set_action() should be of shape (3, 1)
arm_teacher.set_action(action)
# Initialize the GUI
if gui:
    renderer = Renderer()
    time.sleep(1)

for s in range(num_steps):
    # Get the current state
    if s < num_steps*2/3:
        if s % 500 == 0:
            arm_teacher.set_state(initial_state)
    state = arm_teacher.get_state()
    X[s,0:6] = state.T
    action[0] = action_set[s]
    X[s,6:9] = action.T
    # The advance function will simulate the action for 1 time step
    arm_teacher.advance()
    if gui:
        renderer.plot([(arm_teacher, 'tab:blue')])

    # Get the new state after advancing one time step
    new_state = arm_teacher.get_state()
    Y[s,:]=new_state.T
# ---

# DO NOT CHANGE
# Save the collected data in the data.pkl file
data = {'X': X, 'Y': Y}
pickle.dump(data, open( "data.pkl", "wb" ) )

# Part 2. Learning the forward dynamics.

## Training

After the data is collected, you will then need to complete the cell below to use the collected dataset to learn the forward dynamics.

The code already creates the dataset class and loads the dataset with a random 0.8/0.2 train/test split for you. This cell should save the model that it trains. You should use a specific procedure for saving, outlined below.

In machine learning, it is a very good practice to save not only the final model but also the checkpoints, such that you have a wider range of models to choose from. We provide a code snippet for you and for each epoch of your training, you should use it to save the model at that epoch.

```
model_folder_name = f'epoch_{epoch:04d}_loss_{test_loss:.8f}'
if not os.path.exists(os.path.join(model_dir, model_folder_name)):
    os.makedirs(os.path.join(model_dir, model_folder_name))
torch.save(model.state_dict(), os.path.join(model_dir, model_folder_name, 'dynamics.pth'))
```

The output from running this code should be a folder as below:

```
models/
    2023-03-08_23-57-50/
        epoch_0001_loss_0.00032930/
            dynamics.pth
        epoch_0002_loss_0.00009413/
            dynamics.pth   
        ...  
```

You can see that every time you run this cell, a folder whose name is the time you started will be created under `models`. Checkpoints from all epochs will be saved and then the folder name for saving the checkpoint indicates the epoch number and loss on the holdout test set. Recording checkpoints this way allows you to easily pick the model with the smallest loss.


### Important: choosing the best model

Your code should keep track of the checkpoint with the smallest loss on the test set. You should save the path of that checkpoint to the variable `model_path`. An example value of `model_path` could be `models/2023-03-07_20-14-32/epoch_0046_loss_0.00000005/dynamics.pth`. In the evaluation code, the checkpoint from `model_path` will be loaded and evaluated.

You should also download the `dynamic.pth` file to include in your submission.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import os
import numpy as np
import torch
import torch.nn as nn
import argparse
import time
import pickle
np.set_printoptions(suppress=True)
np.random.seed(10)
torch.manual_seed(10)


class DynamicDataset(Dataset):
    def __init__(self, data_file):
        data = pickle.load(open(data_file, "rb" ))
        # X: (N, 9), Y: (N, 6)
        self.X = data['X'].astype(np.float32)
        self.Y = data['Y'].astype(np.float32)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]


class Net(nn.Module):
    # ---
    # Your code goes here
    def __init__(self):
        super(Net, self).__init__()
        # self.fc1 = nn.Linear(9, 1024)
        # self.fc2 = nn.Linear(1024, 512)
        # self.fc3 = nn.Linear(512, 256)
        # self.fc4 = nn.Linear(256, 6)
        self.fc1 = nn.Linear(9, 256)
        self.fc2 = nn.Linear(256, 6)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        # x = torch.relu(self.fc2(x))
        # x = torch.relu(self.fc3(x))
        x = self.fc2(x)
        return x

def train(model):
    model.train()

    # ---
    # Your code goes here
    # ---
    train_loss = 0.0

    for X, Y in train_loader:
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, Y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    return train_loss/len(train_loader)


def test(model):
    model.eval()

    # --
    # Your code goes here
    test_loss = 0
    # ---
    with torch.no_grad():
        for X, Y in test_loader:
            X, Y = X.to(device), Y.to(device)
            outputs = model(X)
            loss = criterion(outputs, Y)
            test_loss += loss.item()
        scheduler.step(test_loss)
    return test_loss/len(test_loader)


# The ratio of the dataset used for testing
split = 0.2

# We are only using CPU, and GPU is not allowed.
device = torch.device("cpu")

dataset = DynamicDataset('data.pkl')
dataset_size = len(dataset)
test_size = int(np.floor(split * dataset_size))
train_size = dataset_size - test_size
train_set, test_set = random_split(dataset, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_set, batch_size = 512, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 512, shuffle=True)

# The name of the directory to save all the checkpoints
timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
model_dir = os.path.join('models', timestr)

# Keep track of the checkpoint with the smallest test loss and save in model_path
model_path = None

model = Net()
model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.8, patience=5, verbose=True, min_lr=1e-4)
best_test_loss = float('inf')
epochs = 300
for epoch in range(1, 1 + epochs):
    # ---
    # Your code goes here
    # ---
    train_loss = train(model)
    test_loss = test(model)

    # Save the model if test loss is minimum
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        model_folder_name = f'epoch_{epoch:04d}_loss_{best_test_loss:.8f}'
        if not os.path.exists(os.path.join(model_dir, model_folder_name)):
            os.makedirs(os.path.join(model_dir, model_folder_name))
        model_path = os.path.join(model_dir, model_folder_name, 'dynamics.pth')
        torch.save(model.state_dict(), os.path.join(model_dir, model_folder_name, 'dynamics.pth'))

    print(f"Epoch {epoch}/{epochs}, Train Loss: {train_loss:.8f}, Test Loss: {test_loss:.8f}")


Epoch 1/300, Train Loss: 0.00354213, Test Loss: 0.00005007
Epoch 2/300, Train Loss: 0.00003495, Test Loss: 0.00002380
Epoch 3/300, Train Loss: 0.00003251, Test Loss: 0.00001483
Epoch 4/300, Train Loss: 0.00004229, Test Loss: 0.00004096
Epoch 5/300, Train Loss: 0.00004022, Test Loss: 0.00005798
Epoch 6/300, Train Loss: 0.00004100, Test Loss: 0.00001300
Epoch 7/300, Train Loss: 0.00003785, Test Loss: 0.00001386
Epoch 8/300, Train Loss: 0.00003395, Test Loss: 0.00003340
Epoch 9/300, Train Loss: 0.00003549, Test Loss: 0.00002510
Epoch 10/300, Train Loss: 0.00003637, Test Loss: 0.00001784
Epoch 11/300, Train Loss: 0.00003835, Test Loss: 0.00006205
Epoch 00012: reducing learning rate of group 0 to 8.0000e-03.
Epoch 12/300, Train Loss: 0.00003702, Test Loss: 0.00011735
Epoch 13/300, Train Loss: 0.00000780, Test Loss: 0.00000911
Epoch 14/300, Train Loss: 0.00002560, Test Loss: 0.00001964
Epoch 15/300, Train Loss: 0.00002528, Test Loss: 0.00002646
Epoch 16/300, Train Loss: 0.00002379, Test Loss

## Prediction

After you are done with training, you need to complete the cell below to load the saved checkpoint (in function init_model) and then use it to predict the new state given the current state and action (in function dynamics_step). Please do not modify the arguments to those functions, even though you might not use all of them.

In [ ]:
from arm_dynamics_base import ArmDynamicsBase
class ArmDynamicsStudent(ArmDynamicsBase):
    def init_model(self, model_path, num_links, device, time_step):
        # ---
        # Your code hoes here
        # Initialize the model loading the saved model from provided model_path
        checkpoint = torch.load(model_path)
        self.model = Net()
        self.model.load_state_dict(checkpoint)
        # ---
        self.model_loaded = True

    def dynamics_step(self, state, action, dt):
        if self.model_loaded:
            # ---
            # Your code goes here
            # Use the loaded model to predict new state given the current state and action
            # Output should be an array of shape (6,1)
            state = np.array(state).reshape(1, -1)
            action = np.array(action).reshape(1, -1)
            x = torch.from_numpy(np.concatenate((state, action),axis=1)).float()
            new_state = self.model(x).detach().numpy().reshape(6,1)
            return new_state
            # ---
        else:
            return state

# Evaluation and Grading

The total number of points for this project is 15. There are 3 types of tests, each is worth 5 points.

**For each type, there are 50 tests.** For each test, you get a score of 1, 0.5, or 0. Your final grade for each type is the averaged score across 50 tests * 5.

- *Type 1*: for each test, a constant torque randomly sampled from [-1.5Nm, 1.5Nm] is applied to the first joint of the arm for 5 seconds. If the MSE (Mean Squred Error) between the predicted arm state (arm_student) and the ground truth arm state (arm_teacher) is < 0.0005, you get score 1 for this test. If 0.0005 <= MSE < 0.008, you get score 0.5 for this test. Otherwise you get 0.
- *Type 2*: for each test, a torque that linearly increases from 0 to a random torque in [0.5Nm, 1.5Nm] is applied to the first joint of the arm for 5 seconds. If MSE < 0.0005, you get score 1 for this test. If 0.0005 <= MSE < 0.008, you get score 0.5 for this test. Otherwise you get 0.
- *Type 3*: for each test, one torque is applied for the first 2.5 seconds and another torque is applied for the remaining 2.5 seconds. Both torques are sampled from [-1Nm, 1Nm]. If MSE < 0.015, you get score 1 for this test. If 0.015 <= MSE < 0.05, you get score 0.5 for this test. Otherwise you get 0.


In [ ]:
# DO NOT CHANGE
# Set up grading

# Make sure model_path is correctly set
print(model_path)

import importlib
import score
importlib.reload(score)

# Create the teacher arm
dynamics_teacher = ArmDynamicsTeacher(
    num_links=3,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01
)
arm_teacher = Robot(dynamics_teacher)

# Create the student arm
dynamics_student = ArmDynamicsStudent(
    num_links=3,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01
)
if model_path is not None:
  dynamics_student.init_model(model_path, num_links=3, time_step=0.01, device=torch.device('cpu'))
arm_student = Robot(dynamics_student)

models/2023-03-31_21-26-37/epoch_0289_loss_0.00000018/dynamics.pth


In [ ]:
# DO NOT CHANGE

# Test on randomly sampled torques from [-1.5, 1.5]
score.score_random_torque(arm_teacher, arm_student, gui=False)


----------------------------------------
TEST 1 (Torque = 0.8139619298002381 Nm)

average mse: 4.364173899886707e-05
Score: 1/1
----------------------------------------


----------------------------------------
TEST 2 (Torque = -1.4377441519217955 Nm)

average mse: 0.00021682444284426854
Score: 1/1
----------------------------------------


----------------------------------------
TEST 3 (Torque = 0.4009447047788264 Nm)

average mse: 3.053238956536081e-05
Score: 1/1
----------------------------------------


----------------------------------------
TEST 4 (Torque = 0.7464116476158358 Nm)

average mse: 4.332768086075701e-05
Score: 1/1
----------------------------------------


----------------------------------------
TEST 5 (Torque = -0.004478963092228838 Nm)

average mse: 2.2593260048773116e-05
Score: 1/1
----------------------------------------


----------------------------------------
TEST 6 (Torque = -0.825610063407457 Nm)

average mse: 5.053348721534322e-05
Score: 1/1
----------

In [ ]:
# DO NOT CHANGE

# Test on torques that linearly increase from 0 to a random number from [0.5, 1.5]
score.score_linear_torques(arm_teacher, arm_student, gui=False)


----------------------------------------
TEST 1 (Torque 0 -> 1.2713206432667459 Nm)

average mse: 2.0783788695682345e-05
Score: 1/1
----------------------------------------


----------------------------------------
TEST 2 (Torque 0 -> 0.5207519493594015 Nm)

average mse: 2.177386135103515e-05
Score: 1/1
----------------------------------------


----------------------------------------
TEST 3 (Torque 0 -> 1.1336482349262753 Nm)

average mse: 2.2998358634083333e-05
Score: 1/1
----------------------------------------


----------------------------------------
TEST 4 (Torque 0 -> 1.2488038825386119 Nm)

average mse: 2.129054544623012e-05
Score: 1/1
----------------------------------------


----------------------------------------
TEST 5 (Torque 0 -> 0.9985070123025904 Nm)

average mse: 2.474392710144983e-05
Score: 1/1
----------------------------------------


----------------------------------------
TEST 6 (Torque 0 -> 0.7247966455308477 Nm)

average mse: 2.3068141466337143e-05
Score:

In [ ]:
# DO NOT CHANGE

# Test on one torque applied to the first 2.5s and another torque applied to the second 2.5s
# Both torques are sampled from [-1, 1]
score.score_two_torques(arm_teacher, arm_student, gui=False)


----------------------------------------
TEST 1 (Torque 1 = 0.542641286533492 Nm,  Torque 2 = -0.21494151210682544 Nm)

average mse: 5.926688126407707e-05
Score: 1/1
----------------------------------------


----------------------------------------
TEST 2 (Torque 1 = -0.958496101281197 Nm,  Torque 2 = -0.8130792508826994 Nm)

average mse: 6.0260323482981045e-05
Score: 1/1
----------------------------------------


----------------------------------------
TEST 3 (Torque 1 = 0.26729646985255084 Nm,  Torque 2 = 0.6422113156738569 Nm)

average mse: 2.5565318523693566e-05
Score: 1/1
----------------------------------------


----------------------------------------
TEST 4 (Torque 1 = 0.4976077650772237 Nm,  Torque 2 = -0.6976959607148723 Nm)

average mse: 0.00015902145399806247
Score: 1/1
----------------------------------------


----------------------------------------
TEST 5 (Torque 1 = -0.0029859753948191514 Nm,  Torque 2 = -0.23177110261560085 Nm)

average mse: 2.5906000497991692e-05

# Other Requirements and Hints

- Training time: This project requires more training than the previous projects. But less than a hundred epochs of training (<= 25 mins) should suffice for achieving the full points. Again, the shorter your model training time is the better.
- Dataset: Choosing the right policy to collect datasets for this project is important. You need to think about how to do it properly so that your trained model will pass the tests successfully. It is in general very hard to learn the ground truth forward dynamics completely (that works for any distribution of actions), and during testing small errors can accumulate, leading to drastic failure in the end. You might want to try overfitting on the test cases, to begin with. Make sure that your dataset is less than 100 Mb, which is pretty much sufficient for achieving full marks. Collecting datasets can be time-consuming and you could parallelize this process for some speed-up using [ray](https://www.ray.io/). Make sure your data collection takes <= 25 mins.
- NO GPU: No GPU is required or allowed for this assignment and we will test your code without GPUs.
- Loss Function: This is essentially a regression problem so think about what losses are suitable for regression.
- Optimizer: While it is possible to use a simple optimizer to achieve the desired accuracy, the training time can be quite high. There exists a number of optimizers implemented in PyTorch that have much faster convergence.
- Seeding. Please use seeding in your code to make sure your results are reproducible.